In [1]:
import requests
import json
import csv
import os
import datetime
import pandas as pd
import logging
import boto3
from botocore.exceptions import ClientError

# Bitmex
https://www.bitmex.com/api/explorer/#/

use XBTUSD as currency

In [139]:
r = requests.get('https://www.bitmex.com/api/v1/stats/history?')
if r.status_code != 200:
    print(r.status_code)
r.json()

[{'date': '2014-11-22T00:00:00.000Z',
  'rootSymbol': 'XBT',
  'currency': 'XBt',
  'volume': 2,
  'turnover': 0},
 {'date': '2014-11-22T00:00:00.000Z',
  'rootSymbol': 'Total',
  'currency': 'XBt',
  'volume': None,
  'turnover': 0},
 {'date': '2014-11-23T00:00:00.000Z',
  'rootSymbol': 'XBT',
  'currency': 'XBt',
  'volume': 2,
  'turnover': 0},
 {'date': '2014-11-23T00:00:00.000Z',
  'rootSymbol': 'Total',
  'currency': 'XBt',
  'volume': None,
  'turnover': 0},
 {'date': '2014-11-24T00:00:00.000Z',
  'rootSymbol': 'XBT',
  'currency': 'XBt',
  'volume': 30315,
  'turnover': 0},
 {'date': '2014-11-24T00:00:00.000Z',
  'rootSymbol': 'Total',
  'currency': 'XBt',
  'volume': None,
  'turnover': 0},
 {'date': '2014-11-25T00:00:00.000Z',
  'rootSymbol': 'XBT',
  'currency': 'XBt',
  'volume': 49220,
  'turnover': 0},
 {'date': '2014-11-25T00:00:00.000Z',
  'rootSymbol': 'Total',
  'currency': 'XBt',
  'volume': None,
  'turnover': 0},
 {'date': '2014-11-26T00:00:00.000Z',
  'rootSymbol'

## 1. Download historical data

In [92]:
now = datetime.datetime.now().isoformat()
bitmex_params = [#('funding',{'symbol':'XBTUSD','count':'100','reverse':'false','start':'0','endTime':now}),    # funding_rate
                 #('liquidation',{}),
                 ('stats/history',{}),    # volume
                 ('stats',{}),    # openInterest, openValue "rootSymbol" -- call "XBT" and "ETH"
                ]

# Initialise historical data. This will not overwrite files of the same name in the target folder so be sure to delete them first if they exist.
def get_historical_bitmex(bitmex_params):
    sample = 'sample'
    for item in bitmex_params:
        endpoint, params_dict = item
        filepath = 'data/Bitmex/' + endpoint + '.csv'
        
        if os.path.isfile(filepath):
            print('Please delete or rename previous file of same name.')
            raise
        
        while sample!='':
            api_call_url = 'https://www.bitmex.com/api/v1/' + endpoint + '?_format=csv'
            # Check if any parameters are applied
            if bool(params_dict):
                for param in params_dict:
                    api_call_url += '&' + param + '=' + params_dict[param] 
            r = requests.get(api_call_url)
            if r.status_code != 200:
                print(r.status_code)
                return
            csv_list = r.text.split('\n')
            
            # Create a new file only if one does not exist, and append the headers
            if not os.path.isfile(filepath):
                with open(filepath, 'w') as csv_file:
                    for row in csv_list[0]:
                        csv_file.write(row)
            
            with open(filepath, 'a') as csv_file:
                # Append the data, but skip the headers
                for row in csv_list[1:]:
                    csv_file.write('\n'+row)
            
            params_dict['start'] = str(int(params_dict['start']) + int(params_dict['count']))
            sample = r.text[:5]

In [ ]:
get_historical_bitmex(bitmex_params)

## 2. Update data

In [121]:
# Ref: https://www.bitmex.com/api/explorer/
# Parameters for each API call. All variables must be in string format
bitmex_params = [('funding',{'symbol':'XBTUSD','count':'100','reverse':'false','start':'0'}),
                 #('liquidation',{}),
                ]

def update_bitmex(bitmex_params):
    # Update data from timestamp of last entry in file
    for item in bitmex_params:
        endpoint, params_dict = item
        filepath = 'data/Bitmex/' + endpoint + '.csv'
        api_call_url = 'https://www.bitmex.com/api/v1/' + endpoint + '?_format=csv'
        
        # Check if any parameters are applied
        if bool(params_dict):
            
            # Check for timestamp of last entry
            timestamp_df = pd.read_csv(filepath, usecols=['timestamp'])
            params_dict['startTime'] = timestamp_df.values[-1][0]
            params_dict['start'] = '1'
            
            for param in params_dict:
                api_call_url += '&' + param + '=' + params_dict[param] 

        r = requests.get(api_call_url)
        if r.status_code != 200:
            print(r.status_code)
            break
        
        csv_list = r.text.split('\n')
        print(csv_list)
        with open(filepath, 'a') as csv_file:
            # Append the data, but skip the headers
            for row in csv_list[1:]:
                csv_file.write(row + '\n')

In [122]:
update_bitmex(bitmex_params)

['"timestamp","symbol","fundingInterval","fundingRate","fundingRateDaily"', '"2019-10-03T12:00:00.000Z","XBTUSD","2000-01-01T08:00:00.000Z",0.0001,0.00030000000000000003', '"2019-10-03T20:00:00.000Z","XBTUSD","2000-01-01T08:00:00.000Z",-0.000133,-0.00039900000000000005', '"2019-10-04T04:00:00.000Z","XBTUSD","2000-01-01T08:00:00.000Z",-0.000183,-0.000549']


# Coinmetrics
https://docs.coinmetrics.io/api/v2/

## a) Get historical data

In [146]:
# Get full data
metrics = 'AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMVRVCur,CapMrktCurUSD,CapRealUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,FeeMedUSD,FeeTotNtv,FeeTotUSD,IssContNtv,IssContPctAnn,IssContUSD,IssTotNtv,IssTotUSD,NVTAdj,NVTAdj90,PriceBTC,PriceUSD,ROI1yr,ROI30d,SplyCur,TxCnt,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d'
r = requests.get('https://community-api.coinmetrics.io/v2/assets/btc/metricdata.csv?metrics='+ metrics)
if r.status_code != 200:
    print(r.status_code)
    raise

In [147]:
csv_list = r.text.split('\n')
with open('data/Coinmetrics/Coinmetrics_btc.csv', 'w') as csv_file:
    for row in csv_list:
        csv_file.write(row)

## b) Update daily

In [189]:
# Get timestamp of last entry
filepath = 'data/Coinmetrics/Coinmetrics_btc.csv'
timestamp_df = pd.read_csv(filepath, usecols=['time'])
timestamp_df = timestamp_df.astype('datetime64')
timestamp_df.values[-1][0]+1
start_time = str(timestamp_df.values[-1][0]+10000000)
start = '&start=' + str(start_time)[:23] + 'Z'

metrics = 'AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMVRVCur,CapMrktCurUSD,CapRealUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,FeeMedUSD,FeeTotNtv,FeeTotUSD,IssContNtv,IssContPctAnn,IssContUSD,IssTotNtv,IssTotUSD,NVTAdj,NVTAdj90,PriceBTC,PriceUSD,ROI1yr,ROI30d,SplyCur,TxCnt,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d' #,TxTfr
r = requests.get('https://community-api.coinmetrics.io/v2/assets/btc/metricdata.csv?metrics='+ metrics + start)
if r.status_code != 200:
    print(r.status_code)
    raise
csv_list = r.text.split('\n')
with open(filepath, 'a') as csv_file:
    for row in csv_list[1:]:
        csv_file.write(row)

# Quandl
https://www.quandl.com/tools/python

# Coinfarm

In [1]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup

In [3]:
# create an HTML Session object
session = HTMLSession()
# Use the object above to connect to needed webpage
resp = session.get("https://coinfarm.online/position/position_realtime.asp")
# Run JavaScript code on webpage
resp.html.render(sleep=5)
soup = BeautifulSoup(resp.html.html, 'html.parser')

RuntimeError: Cannot use HTMLSession within an existing event loop. Use AsyncHTMLSession instead.

In [ ]:
soup.body.center.find('table', {'style':"background:#253440;"},
                      recursive=False).find('tbody', recursive=False).find('tr',
                                                                           recursive=False).find('td',{"style":"background:url('/public/bg_2.png') no-repeat ;"}).table.tbody.find_all('tr',recursive=False)[6]
# row = td_tag.table.tbody.find_all('tr',recursive=False)[6]
# header = row.find_all('td', recursive=False)[0].table.tbody.tr

In [11]:
with open('coinfarm.txt', 'r', encoding='utf-8') as html_file:
    html_text = str(html_file)

In [12]:
html_text

"<_io.TextIOWrapper name='coinfarm.txt' mode='r' encoding='utf-8'>"

In [13]:
coinfarm_bs = BeautifulSoup(html_text, 'html.parser')

In [14]:
coinfarm_bs.body.center.find('table', 
                              {'style':"background:#253440;"}, 
                              recursive=False).find('tbody', 
                                                    recursive=False).find('tr',
                                                                          recursive=False).find('td',
                                                                                                recursive=False)
#/html/body/center/table/tbody/tr/td/table[2]/tbody/tr/td/table/tbody/tr[7]/td/table/tbody/tr/td[3]/span

AttributeError: 'NoneType' object has no attribute 'center'

# Glassnode

In [ ]:
variables = ['Sopr', 'aSopr', 'Cdd', 'AverageDormancy'. 'AverageDormancySupplyAdjusted']
for var in variables:
    r = requests.get('https://studio.glassnode.com/metrics?a=BTC&m=indicators.' + var)
    if r.status_code != 200:
        print(r.status_code)
        raise
    r.text

# Deribit
https://docs.deribit.com/v2/?python#methods

In [166]:
r.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [81]:
with open('data.json', 'w') as f:
    json.dump(data, f)

In [82]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [83]:
# Let's use Amazon S3
s3 = boto3.resource('s3')
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

your-s3-bucket-name-deeloon


In [84]:
data = open('data.json', 'rb')
s3.Bucket('your-s3-bucket-name-deeloon').put_object(Key='data.json', Body=data)

s3.Object(bucket_name='your-s3-bucket-name-deeloon', key='data.json')